In [1]:
from qiskit.algorithms import VQE
from qiskit.opflow import (
    AerPauliExpectation,
    Gradient,
    I,
    MatrixExpectation,
    PauliExpectation,
    PauliSumOp,
    PrimitiveOp,
    TwoQubitReduction,
    X,
    Z,
)
from qiskit.algorithms.optimizers import (
    CG,
    COBYLA,
    L_BFGS_B,
    P_BFGS,
    QNSPSA,
    SLSQP,
    SPSA,
    TNC,
)
from qiskit.circuit.library import EfficientSU2, RealAmplitudes, TwoLocal
from qiskit.utils import QuantumInstance, algorithm_globals, has_aer


from qiskit_braket_plugin import AWSBraketProvider, BraketLocalBackend



In [2]:
provider = AWSBraketProvider()
local_simulator = BraketLocalBackend(name="default")
# local_simulator = provider.backends(local=True)
local_simulator

BraketBackend[default]

In [21]:
seed = 50

h2_op = (
            -1.052373245772859 * (I ^ I)
            + 0.39793742484318045 * (I ^ Z)
            - 0.39793742484318045 * (Z ^ I)
            - 0.01128010425623538 * (Z ^ Z)
            + 0.18093119978423156 * (X ^ X)
        )
h2_energy = -1.85727503

ryrz_wavefunction = TwoLocal(rotation_blocks=["ry", "rz"], entanglement_blocks="cz")
ry_wavefunction = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")

vqe = VQE(
            ansatz=ryrz_wavefunction,
            optimizer=L_BFGS_B(),
            quantum_instance=QuantumInstance(
                local_simulator
            ),
        )

In [22]:
from qiskit import transpile

In [23]:
result = vqe.compute_minimum_eigenvalue(operator=h2_op)

TranspilerError: "Unable to map source basis {('ry', 1), ('measure', 1), ('rz', 1), ('h', 1), ('cz', 2)} to target basis {'ry', 'rx', 'sx', 'z', 'cz', 'y', 'u1', 'u2', 'rz', 'rxx', 'id', 'p', 'tdg', 'rzz', 'u3', 'cp', 'snapshot', 'ccx', 'ryy', 't', 'cx', 'cswap', 's', 'swap', 'x', 'sxdg', 'cy', 'sdg', 'barrier', 'h'} over library <qiskit.circuit.equivalence.EquivalenceLibrary object at 0x7fb1903ffcd0>."

In [28]:
from qiskit import QuantumCircuit

circ = QuantumCircuit(2)
circ.h(0)
circ.cx(0, 1)
circ.measure_all()

circ.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1

In [39]:
circ = TwoLocal(rotation_blocks=["ry", "rz"], entanglement_blocks="cz")
circ.draw()

In [30]:
result = local_simulator.run(circ)

In [32]:
result.result()

Result(backend_name='BraketBackend[default]', backend_version='2', qobj_id='0', job_id='dade2207-13f2-4023-8c00-05ba989d475f', success=JobStatus.DONE, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'00': 516, '11': 508}), status=COMPLETED)])